Dask-ML
====================================================================

__(initial notes are courtesy of the Dask project homepage at ml.dask.org)__

Dask-ML provides scalable machine learning in Python using [Dask](https://dask.org/) alongside popular machine learning libraries like [Scikit-Learn](http://scikit-learn.org/).

The idea is to support Pandas + Scikit style ML for parallel scenarios, with code patterns you're used to:

```python
import dask.dataframe as dd
df = dd.read_parquet('...')
data = df[['age', 'income', 'married']]
labels = df['outcome']

from dask_ml.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(data, labels)
```

How does this work?
-------------------------------------------------------------------------------------------

Modern machine learning algorithms employ a wide variety of techniques. Scaling these requires a similarly wide variety of different approaches. Generally solutions fall into the following three categories:

### Parallelize Scikit-Learn Directly

Scikit-Learn already provides parallel computing on a single machine with [Joblib](http://joblib.readthedocs.io/en/latest/). Dask extends this parallelism to many machines in a cluster. This works well for modest data sizes but large computations, such as random forests, hyper-parameter optimization, and more.

```python
from dask.distributed import Client
import joblib

client = Client()  # Connect to a Dask Cluster

with joblib.parallel_backend('dask'):
    # Your normal scikit-learn code here
```

See [Dask-ML Joblib documentation](https://ml.dask.org/joblib.html) for more information.

*Note that this is an active collaboration with the Scikit-Learn development team. This functionality is progressing quickly but is in a state of rapid change.*

### Reimplement Scalable Algorithms with Dask Array

Some machine learning algorithms are easy to write down as Numpy algorithms. In these cases we can replace Numpy arrays with Dask arrays to achieve scalable algorithms easily. This is employed for [linear models](https://ml.dask.org/glm.html), [pre-processing](https://ml.dask.org/preprocessing.html), and [clustering](https://ml.dask.org/clustering.html).

```python
from dask_ml.preprocessing import Categorizer, DummyEncoder
from dask_ml.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(data, labels)
```

### Partner with other distributed libraries

Other machine learning libraries like XGBoost and TensorFlow already have distributed solutions that work quite well. Dask-ML makes no attempt to re-implement these systems. Instead, Dask-ML makes it easy to use normal Dask workflows to prepare and set up data, then it deploys XGBoost or Tensorflow *alongside* Dask, and hands the data over.

```python
from dask_ml.xgboost import XGBRegressor

est = XGBRegressor(...)
est.fit(train, train_labels)
```

See [Dask-ML + XGBoost](https://ml.dask.org/xgboost.html) or [Dask-ML + TensorFlow](https://ml.dask.org/tensorflow.html) documentation for more information.

Scikit-Learn API[](https://ml.dask.org/#scikit-learn-api "Permalink to this headline")
--------------------------------------------------------------------------------------

In all cases Dask-ML endeavors to provide a single unified interface around the familiar NumPy, Pandas, and Scikit-Learn APIs. Users familiar with Scikit-Learn should feel at home with Dask-ML.

* * *

# Let's try it:

In [ ]:
import dask.dataframe

ddf = dask.dataframe.read_csv('data/diamonds.csv', blocksize=1e6)
ddf

In [ ]:
ddf = ddf.drop(['Unnamed: 0'], axis=1)
ddf

In [ ]:
ddf = ddf.categorize()
ddf

In [ ]:
prepared = dask.dataframe.reshape.get_dummies(ddf)
prepared

Alternatively, we could use scikit-learn style preprocessing steps, though the relevant APIs are still evolving a bit:

```python
from sklearn.pipeline import make_pipeline
from dask_ml.preprocessing import Categorizer, DummyEncoder
from dask_ml.linear_model import LinearRegression

pipe = make_pipeline(
    Categorizer(),
    DummyEncoder()
)

pipe.fit(ddf)

prepared = pipe.transform(ddf)
```

In [ ]:
prepared.head()

In [ ]:
y = prepared.price.to_dask_array(lengths=True)
y

In [ ]:
arr = prepared.drop('price', axis=1).to_dask_array(lengths=True)
arr

In [ ]:
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(arr, y, test_size=0.1)

X_train

In [ ]:
y_train

In [ ]:
from dask_ml.linear_model import LinearRegression

lr = LinearRegression()
lr_model = lr.fit(X_train, y_train)

In [ ]:
lr_model.coef_

In [ ]:
y_predicted = lr_model.predict(X_test)

y_predicted

In [ ]:
y_test

In [ ]:
from dask_ml.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(y_test, y_predicted))

### Sometime very soon...

We'll do an official lab where we try at least one additional algorithm on real data.

Although we can smoke-test algorithms with synthetic data in the current Dask-ML version (1.0.0), that may be a good use of the limited time in a short session such as this.